In [22]:
import torch  # 命令行是逐行立即执行的
content = torch.load('data.pth')
print(content.keys())   # keys()
# 之后有其他需求比如要看 key 为 model 的内容有啥
print(content['feature'])
print(content['label'])


dict_keys(['feature', 'label'])
tensor([[ -1.9408,   4.2829,  -7.0747,  ...,  17.2657,  -2.8689,   4.2029],
        [ -1.2524,   3.3057,  -1.6193,  ...,   9.4712, -10.3804,  -3.0191],
        [  1.1509,  -2.9682,  -9.6701,  ...,  -3.4445,   6.2805,  -0.3120],
        ...,
        [ -1.5408,   2.5173,  -1.8167,  ...,  12.3208,  -1.4063,  -0.0334],
        [ -4.4179,   2.0405, -12.9277,  ...,  12.6393,  -2.1744,  -3.5770],
        [ -3.1111,  -2.0232,  -7.9568,  ...,   7.1998,  -6.0398,   3.6039]])
tensor([5, 0, 4,  ..., 5, 6, 8])


In [23]:
# Import dependencies
import torch
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

# Set the hyperparameters for data creation
NUM_CLASSES = 10
NUM_FEATURES = 256
RANDOM_SEED = 42

# 1. Create multi-class data
# X_blob, y_blob = make_blobs(n_samples=1000,
#     n_features=NUM_FEATURES, # X features
#     centers=NUM_CLASSES, # y labels
#     cluster_std=1.5, # give the clusters a little shake up (try changing this to 1.0, the default)
#     random_state=RANDOM_SEED
# )
#
# # 2. Turn data into tensors
# X_blob = torch.from_numpy(X_blob).type(torch.float)
# y_blob = torch.from_numpy(y_blob).type(torch.LongTensor)
X_blob=content['feature']
y_blob=content['label']

print(X_blob[:5], y_blob[:5])

# 3. Split into train and test sets
X_blob_train, X_blob_test, y_blob_train, y_blob_test = train_test_split(X_blob,
    y_blob,
    test_size=0.2,
    random_state=RANDOM_SEED
)

# 4. Plot data
# plt.figure(figsize=(10, 7))
# plt.scatter(X_blob[:, 0], X_blob[:, 1], c=y_blob, cmap=plt.cm.RdYlBu);

tensor([[ -1.9408,   4.2829,  -7.0747,  ...,  17.2657,  -2.8689,   4.2029],
        [ -1.2524,   3.3057,  -1.6193,  ...,   9.4712, -10.3804,  -3.0191],
        [  1.1509,  -2.9682,  -9.6701,  ...,  -3.4445,   6.2805,  -0.3120],
        [  8.5502,   4.3298, -11.2310,  ...,  -2.5779,  -7.6427,   0.9930],
        [  2.4465,  -2.8796,  -1.0813,  ...,  12.2426,  -7.7521, -14.5718]]) tensor([5, 0, 4, 1, 9])


In [24]:
# Standard PyTorch imports
import torch
from torch import nn

# Make device agnostic code
device =  "cpu"
device

'cpu'

In [25]:
from torch import nn

# Build model
class BlobModel(nn.Module):
    def __init__(self):
        """Initializes all required hyperparameters for a multi-class classification model.

        Args:
            input_features (int): Number of input features to the model.
            out_features (int): Number of output features of the model
              (how many classes there are).
            hidden_units (int): Number of hidden units between layers, default 8.
        """
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            nn.Linear(in_features=64, out_features=10), # how many classes are there?



        )

    def forward(self, x):
        x=self.flatten(x)
        logists=self.linear_layer_stack(x)
        return logists

# Create an instance of BlobModel and send it to the target device
model_4 = BlobModel().to(device)
model_4

BlobModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [26]:
# Create loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_4.parameters(),lr=0.01,momentum=0.9) # exercise: try changing the learning rate here and seeing what happens to the model's performance
# optimizer=torch.optim.Adam(params=model_4.parameters(),lr=0.1)


In [25]:
# print(model_4(X_blob_train.to(device)))

In [30]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc
# Fit the model
torch.manual_seed(42)

# Set number of epochs
epochs = 500

# Put data to target device
X_blob_train, y_blob_train = X_blob_train.to(device), y_blob_train.to(device)
X_blob_test, y_blob_test = X_blob_test.to(device), y_blob_test.to(device)

for epoch in range(epochs):
    ### Training
    model_4.train()

    # 1. Forward pass

    y_logits = model_4(X_blob_train) # model outputs raw logits
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    # print(y_logits)
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_blob_train)
    acc = accuracy_fn(y_true=y_blob_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_4.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model_4(X_blob_test)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      # 2. Calculate test loss and accuracy
      test_loss = loss_fn(test_logits, y_blob_test)
      test_acc = accuracy_fn(y_true=y_blob_test,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.00492, Acc: 99.99% | Test Loss: 0.11910, Test Acc: 97.17%
Epoch: 10 | Loss: 0.00488, Acc: 99.99% | Test Loss: 0.11921, Test Acc: 97.17%
Epoch: 20 | Loss: 0.00483, Acc: 99.99% | Test Loss: 0.11933, Test Acc: 97.17%
Epoch: 30 | Loss: 0.00479, Acc: 99.99% | Test Loss: 0.11945, Test Acc: 97.17%
Epoch: 40 | Loss: 0.00475, Acc: 99.99% | Test Loss: 0.11956, Test Acc: 97.18%
Epoch: 50 | Loss: 0.00470, Acc: 99.99% | Test Loss: 0.11968, Test Acc: 97.18%
Epoch: 60 | Loss: 0.00466, Acc: 99.99% | Test Loss: 0.11980, Test Acc: 97.18%
Epoch: 70 | Loss: 0.00462, Acc: 99.99% | Test Loss: 0.11991, Test Acc: 97.18%
Epoch: 80 | Loss: 0.00458, Acc: 99.99% | Test Loss: 0.12003, Test Acc: 97.17%
Epoch: 90 | Loss: 0.00454, Acc: 99.99% | Test Loss: 0.12014, Test Acc: 97.17%
Epoch: 100 | Loss: 0.00450, Acc: 99.99% | Test Loss: 0.12026, Test Acc: 97.17%
Epoch: 110 | Loss: 0.00446, Acc: 99.99% | Test Loss: 0.12037, Test Acc: 97.17%
Epoch: 120 | Loss: 0.00442, Acc: 99.99% | Test Loss: 0.12049, T

In [31]:
torch.save(model_4.state_dict(), "D:\\files2\\ai\\lab\\AiLabCode\\Proj3\\model8.pth")

In [31]:
torch.save(model_4, "D:\\files2\\ai\\lab\\AiLabCode\\Proj3\\model2.pth")

In [11]:
modelFinal=torch.load("D:\\files2\\ai\\lab\\AiLabCode\\Proj3\\model2.pth")
modelFinal

BlobModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [12]:
torch.save(modelFinal.state_dict(), "D:\\files2\\ai\\lab\\AiLabCode\\Proj3\\model7.pth")

In [13]:
model_dict=modelFinal.load_state_dict(torch.load("D:\\files2\\ai\\lab\\AiLabCode\\Proj3\\model7.pth"))
model_dict

<All keys matched successfully>

In [10]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc
# Fit the model
torch.manual_seed(42)

# Set number of epochs
epochs = 100

# Put data to target device
X_blob_train, y_blob_train = X_blob_train.to(device), y_blob_train.to(device)
X_blob_test, y_blob_test = X_blob_test.to(device), y_blob_test.to(device)

for epoch in range(epochs):
    ### Training
    modelFinal.train()

    # 1. Forward pass

    y_logits = modelFinal(X_blob_train) # model outputs raw logits
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    # print(y_logits)
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_blob_train)
    acc = accuracy_fn(y_true=y_blob_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    modelFinal.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = modelFinal(X_blob_test)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      # 2. Calculate test loss and accuracy
      test_loss = loss_fn(test_logits, y_blob_test)
      test_acc = accuracy_fn(y_true=y_blob_test,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.00241, Acc: 100.00% | Test Loss: 0.12751, Test Acc: 97.19%
Epoch: 10 | Loss: 0.00241, Acc: 100.00% | Test Loss: 0.12751, Test Acc: 97.19%
Epoch: 20 | Loss: 0.00241, Acc: 100.00% | Test Loss: 0.12751, Test Acc: 97.19%
Epoch: 30 | Loss: 0.00241, Acc: 100.00% | Test Loss: 0.12751, Test Acc: 97.19%
Epoch: 40 | Loss: 0.00241, Acc: 100.00% | Test Loss: 0.12751, Test Acc: 97.19%
Epoch: 50 | Loss: 0.00241, Acc: 100.00% | Test Loss: 0.12751, Test Acc: 97.19%
Epoch: 60 | Loss: 0.00241, Acc: 100.00% | Test Loss: 0.12751, Test Acc: 97.19%
Epoch: 70 | Loss: 0.00241, Acc: 100.00% | Test Loss: 0.12751, Test Acc: 97.19%
Epoch: 80 | Loss: 0.00241, Acc: 100.00% | Test Loss: 0.12751, Test Acc: 97.19%
Epoch: 90 | Loss: 0.00241, Acc: 100.00% | Test Loss: 0.12751, Test Acc: 97.19%


In [20]:
import torch
# NB: vmap is only available on nightly builds of PyTorch.
# You can download one at pytorch.org if you're interested in testing it out.
from functorch import vmap
batch_size, feature_size = 3, 5
weights = torch.randn(feature_size, requires_grad=True)

# Note that model doesn't work with a batch of feature vectors because
# torch.dot must take 1D tensors. It's pretty easy to rewrite this
# to use `torch.matmul` instead, but if we didn't want to do that or if
# the code is more complicated (e.g., does some advanced indexing
# shenanigins), we can simply call `vmap`. `vmap` batches over ALL
# inputs, unless otherwise specified (with the in_dims argument,
# please see the documentation for more details).
def model(feature_vec):
    # Very simple linear model with activation
    return feature_vec.dot(weights).relu()

examples = torch.randn(batch_size, feature_size)
result = vmap(model)(examples)
expected = torch.stack([model(example) for example in examples.unbind()])
print(weights)
print(examples)
print(result)
print(expected)

assert torch.allclose(result, expected)

tensor([-0.7905, -0.9058, -0.2607, -0.5465,  2.1174], requires_grad=True)
tensor([[-1.7118,  0.1651,  1.5819,  0.4485,  0.0330],
        [ 1.4503, -0.6936,  0.9967,  0.6131,  0.7764],
        [-0.3029, -1.2753, -0.4758,  2.3839,  0.9157]])
tensor([0.6160, 0.5309, 2.1548], grad_fn=<ReluBackward0>)
tensor([0.6160, 0.5309, 2.1548], grad_fn=<StackBackward0>)


NameError: name 'model_4' is not defined